In [1]:
import pandas as pd
from configparser import ConfigParser
import psycopg2


/tmp/ipykernel_7448/2200365723.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def load_config(filename='database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    # get section, default to postgresql
    config = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            config[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return config

def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

## Parsing data

In [21]:
df_train = pd.read_csv("../dataset/NYC/original/NYC_train.csv")
df_test = pd.read_csv("../dataset/NYC/original/NYC_test.csv")
df_val = pd.read_csv("../dataset/NYC/original/NYC_val.csv")

In [24]:
df_train = df_train[["POI_id", "POI_catname", "latitude", "longitude"]]
df_test = df_test[["POI_id", "POI_catname", "latitude", "longitude"]]
df_val = df_val[["POI_id", "POI_catname", "latitude", "longitude"]]

In [25]:
df_train

,POI_id,POI_catname,latitude,longitude
0,49bbd6c0f964a520f4531fe3,Arts & Crafts Store,40.719810,-74.002581
1,4a43c0aef964a520c6a61fe3,Bridge,40.606800,-74.044170
2,4c5cc7b485a1e21e00d35711,Home (private),40.716162,-73.883070
3,4ce1863bc4f6a35d8bd2db6c,Home (private),40.619151,-74.035888
4,4c0ab56f7e3fc9288c1df482,Mobile Phone Shop,40.741191,-73.989663
...,...,...,...,...
83223,50170a53e4b07ac9f310b944,Coffee Shop,40.709272,-74.005450
83224,4a9fc4fbf964a520493d20e3,Train Station,40.747582,-74.171888
83225,4d056fc57d9ba35d1c446b23,Building,40.760184,-73.969324
83226,4ae1f681f964a520318921e3,Clothing Store,40.753716,-73.980947


In [26]:
df_test

,POI_id,POI_catname,latitude,longitude
0,4fc7f3dfe4b0896b9f093264,Moving Target,40.813099,-74.001801
1,50c85b8de4b090305763bf7e,Historic Site,40.755385,-73.983650
2,4c3725790a71c9b60a3f3ec9,Neighborhood,40.753696,-73.988441
3,4a7c5c69f964a5205dec1fe3,Food Truck,40.753889,-73.987384
4,4e18dbedae6092c27656cead,Residential Building (Apartment / Condo),40.703220,-73.943671
...,...,...,...,...
10369,4e43c5c752b18fcc7991cffa,Deli / Bodega,40.832593,-73.915289
10370,4f79a81ae4b09489387bafb6,Deli / Bodega,40.832612,-73.915352
10371,4459d2f1f964a520d7321fe3,Airport,40.689685,-74.179387
10372,4ea54ec1b6347b9968f892b2,Café,40.807916,-74.208890


In [27]:
df_val

,POI_id,POI_catname,latitude,longitude
0,4fa5cedbe4b038e6a2b5ed1c,Bar,40.965172,-74.062746
1,3fd66200f964a520bce31ee3,Diner,40.752188,-73.993585
2,4b0737c3f964a520e4f922e3,Subway,40.756506,-73.988028
3,42911d00f964a520f5231fe3,Train Station,40.750795,-73.993576
4,4a70a76cf964a52017d81fe3,Office,40.738155,-74.001232
...,...,...,...,...
10334,4bec4056fd60a5932dba3af1,Office,40.910850,-74.031467
10335,4d2f32b573e5a0935c5158cf,Building,40.969114,-73.880592
10336,4d0304fc54d0236ac1a2e6d5,Hotel,40.965371,-74.062815
10337,4b857b2ff964a520b66031e3,City,40.817222,-74.003419


In [28]:
df_all = pd.concat([df_train, df_test, df_val])
df_all

,POI_id,POI_catname,latitude,longitude
0,49bbd6c0f964a520f4531fe3,Arts & Crafts Store,40.719810,-74.002581
1,4a43c0aef964a520c6a61fe3,Bridge,40.606800,-74.044170
2,4c5cc7b485a1e21e00d35711,Home (private),40.716162,-73.883070
3,4ce1863bc4f6a35d8bd2db6c,Home (private),40.619151,-74.035888
4,4c0ab56f7e3fc9288c1df482,Mobile Phone Shop,40.741191,-73.989663
...,...,...,...,...
10334,4bec4056fd60a5932dba3af1,Office,40.910850,-74.031467
10335,4d2f32b573e5a0935c5158cf,Building,40.969114,-73.880592
10336,4d0304fc54d0236ac1a2e6d5,Hotel,40.965371,-74.062815
10337,4b857b2ff964a520b66031e3,City,40.817222,-74.003419


In [29]:
df_all.drop_duplicates(subset=["POI_id"], inplace=True)
df_all

,POI_id,POI_catname,latitude,longitude
0,49bbd6c0f964a520f4531fe3,Arts & Crafts Store,40.719810,-74.002581
1,4a43c0aef964a520c6a61fe3,Bridge,40.606800,-74.044170
2,4c5cc7b485a1e21e00d35711,Home (private),40.716162,-73.883070
3,4ce1863bc4f6a35d8bd2db6c,Home (private),40.619151,-74.035888
4,4c0ab56f7e3fc9288c1df482,Mobile Phone Shop,40.741191,-73.989663
...,...,...,...,...
5768,4e8e24c97ee6c06365762ab6,Bar,40.723503,-74.002845
8029,50c811dbe4b0d585092b704f,Other Great Outdoors,40.755905,-73.986869
8139,50c7104fe4b0860fb5620c72,Concert Hall,40.769634,-73.994346
8798,4d0382fa30a58cfa181399e7,Other Nightlife,40.754279,-73.987083


In [31]:
df_all.to_csv("../dataset/NYC/original/all_pois.csv")

## Adicionando no Banco

In [56]:
config = load_config()
conn = connect(config)
cursor = conn.cursor()

Connected to the PostgreSQL server.


In [50]:
cursor.execute("""CREATE TABLE poi (
               id VARCHAR(255) PRIMARY KEY,
               category VARCHAR(255),
               latitude FLOAT8,
               longitude FLOAT8);
""")


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [55]:
conn.commit()

cursor.close()
conn.close()

In [57]:
for index, row in df_all.iterrows():
    cursor.execute(f"""INSERT INTO poi (id, category, latitude, longitude)
                   VALUES ('{row['POI_id']}', '{row['POI_catname']}', {row['latitude']}, {row['longitude']})

    """)

In [58]:
conn.commit()

cursor.close()
conn.close()